In [2]:
pip install keras-tcn

  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 2.0
    Uninstalling flatbuffers-2.0:
      Successfully uninstalled flatbuffers-2.0
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-nightly 2.10.0.dev20220617 requires flatbuffers>=2.0, but you have flatbuffers 1.12 which is incompatible.


In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

from sklearn.model_selection import train_test_split
from tcn import compiled_tcn

# 데이터 로드

In [2]:
curr_dir = os.getcwd()

forders_tr = os.listdir(curr_dir+"\\data")

data = pd.read_csv(os.path.join(curr_dir+"\\data\\")+forders_tr[0])
data = data.drop(data.columns[0], axis=1)
data

submission = pd.read_csv('2-1_검증데이터셋.csv')
submission1 = submission.copy()          

# 전처리

In [121]:
soil = pd.read_csv('OBS_AAOS_TIM_20220802210224.csv', encoding='cp949')       # 외부 데이터 로드
stn = list(set(soil['지점'].values))

soil_df1 = []      # 일시 포함된 데이터 프레임
for i in range(len(stn)):
    df = soil[soil['지점']==stn[i]]
    df = df.reset_index()
    df = df.drop('index', axis=1)
    soil_df1.append(df)

    
soil_df = []        # 일시가 제외된 데이터 프레임 (추합을 위해)
for i in range(len(stn)):
    df = soil[soil['지점']==stn[i]]
    df = df.reset_index()
    df = df.drop('index', axis=1)
    df = df.drop(['지점', '지점명', '일시'], axis=1)   
    soil_df.append(df) 
  
day = pd.date_range('2021-11-26', periods=95*24+20*24, freq='1H')    # 21.11.26~22.03.20
day
day1 = pd.DataFrame(day)
day1.columns = ['일자']
day1
day1.to_csv('day_real.csv', index=False)


for i in range(len(stn)):
    df = soil[soil['지점']==stn[i]]
    df.to_csv(os.path.join(str(stn[i])+'.csv'), index=False)   # 관측소마다.csv 파일로 저장

In [251]:
# 관측소마다 일시(일자) 통합을 위해 'day_real'파일과 위에서 저장된 모든 관측소에 대해 엑셀 VLOOKUP 함수 이용하여 
# 중간에 비어 있는 일시를 채워주는 작업 후, 각 파일을 _1.csv 파일로 저장

soil_df2 = []
for i in range(len(stn)):
    df = pd.read_csv(os.path.join(str(stn[i])+'_1.csv'), encoding='cp949')
    df = df.drop('일자', axis=1)
    df = df.fillna(0)
    soil_df2.append(df)



soil_df3 = soil_df2[0].copy()
for i in range(len(soil_df[0].columns)):
    soil_df2[0][soil_df2[0].columns[i]] = 0

# 각 관측소마다 평균 내기 (추합)
for i in range(1, len(soil_df2)):
    soil_df3 = soil_df3 + soil_df2[i]
soil_df3 = soil_df3/len(soil_df2)
soil_df3['일자'] = pd.date_range('2021-11-26', periods=95*24+20*24, freq='1H')


# 제공 데이터와의 통합을 위해 제공 데이터에 없는 일시를 외부 데이터에서 제외시킴
d1 = soil_df3[soil_df3['일자'] == '2021-12-22 0:00'].index[0]
for i in range(d1, d1+24*3):
    soil_df3 = soil_df3.drop(i,axis=0)

d2 = soil_df3[soil_df3['일자'] == '2022-03-01 0:00'].index[0]
for i in range(d2, d2+24*3):
    soil_df3 = soil_df3.drop(i,axis=0)
    
d3 = soil_df3[soil_df3['일자'] == '2022-03-14 0:00'].index[0]
for i in range(d3, d3+24):
    soil_df3 = soil_df3.drop(i,axis=0)
    
d4 = soil_df3[soil_df3['일자'] == '2022-03-16 0:00'].index[0]
for i in range(d4, d4+24*2):
    soil_df3 = soil_df3.drop(i,axis=0)

soil_df3 = soil_df3.reset_index()
soil_df3 = soil_df3.drop('index', axis=1)

# 제공 데이터와 외부 데이터 통합 (시간 데이터 제외 모든 열)
col = ['smart_farm.insolation', 'smart_farm.out_tmperature',\
       'smart_farm.out_humidity', 'smart_farm.wind_speed',\
       'smart_farm.wind_direction', 'smart_farm.in_tmperature',\
       'smart_farm.in_humidity', 'smart_farm.shield_light_h',\
       'smart_farm.shield_tmperature_h', 'smart_farm.shield_energy_h',\
       'smart_farm.shield_energy_v', 'smart_farm.exhaust_fan',\
       'smart_farm.ceiling', 'smart_farm.floating_fan',\
       'smart_farm.fan_coil_b_site',\
       'smart_farm.ventilation_temperature_control',\
       'smart_farm.heating_temperature_set_up']          

data_ = pd.DataFrame()
for k in tqdm(range(len(col))):
    h = []
    for i in range(int(len(data)/60)):      # 분 -> 시간 단위 바꾸기 위해
        h.append(np.mean(data[col[k]][60*i:60*(i+1)]))
    data_[col[k]] = h
    
h1 = []
for i in range(int(len(data)/60)):
    h1.append(np.sum(data['smart_farm.heat_supply'][60*i:60*(i+1)]))
data_['smart_farm.heat_supply'] = h1



real_data = pd.concat([soil_df3, data_], axis=1)
real_data1 = pd.concat([real_data[real_data.columns[4]], real_data[real_data.columns[:4]], real_data[real_data.columns[5:]]], axis=1)
real_data1 = real_data1.reset_index()
real_data1 = real_data1.drop('index', axis=1)
real_data1 = real_data1.dropna(subset = ['일자'])
real_data1.to_csv('real_data_통합.csv', index=False)

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:05<00:00,  2.86it/s]


In [13]:
# 위에서 저장한 'real_data_통합' 파일과 'day_real' 파일 이용하여
# 엑셀 VLOOKUP 함수 이용하여 21.11.26부터 22.03.20까지 비어 일자 채운 후, 다시 'real_data_결측' 파일로 저장

pre = pd.read_csv('real_data_결측.csv', encoding='cp949')
pre1 = pre.interpolate(method ='linear')      
pre1 = pre1.dropna(subset = ['일자'])
pre1.to_csv('real_data_최종.csv', index=False)  # 최종 전처리 파일(결측치 제거한)

real1 = pd.read_csv('real_data_최종.csv', encoding='utf-8')
real1

# 자른 이유? 20개를 추출하기 위해 일자 슬라이싱이 필요함.
training = real1[:-504]
testing = real1[-504:]
testing = testing.reset_index()
testing = testing.drop('index', axis=1)

# 모델 1

In [27]:
X = training.drop(['10CM 정시 토양수분(%)', '20CM 정시 토양수분(%)', '30CM 정시 토양수분(%)',\
       '50CM 정시 토양수분(%)','일자','smart_farm.heat_supply'], axis=1)
y = training[['smart_farm.heat_supply']].values

training_data, test_data, y_tr, y_te = train_test_split(X, y, test_size=0.3, shuffle=False)  

validation = testing.drop(['10CM 정시 토양수분(%)', '20CM 정시 토양수분(%)', '30CM 정시 토양수분(%)',\
       '50CM 정시 토양수분(%)','일자','smart_farm.heat_supply'], axis=1)


y_tr1 = y_tr.flatten()
y_te1 = y_te.flatten()

x_train = []
y_train = []

for i in range(24, training_data.shape[0]):
  x_train.append(training_data[i-24:i])
  y_train.append(y_tr1[i])
    
    
x_test = []
y_test = []

for i in range(24, test_data.shape[0]):
  x_test.append(test_data[i-24:i])
  y_test.append(y_te1[i])
x_train, y_train = np.array(x_train), np.array(y_train)
x_test, y_test = np.array(x_test), np.array(y_test)


x_validation = []
for i in range(24, validation.shape[0]):
  x_validation.append(validation[i-24:i])
x_validation = np.array(x_validation)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(x_validation.shape)


model11 = compiled_tcn(return_sequences=False,
                      num_feat=x_test.shape[2],
                      nb_filters=50,    
                      num_classes=0,
                      kernel_size=3,     
                      dilations=[2 ** i for i in range(9)],
                      nb_stacks=1,
                      max_len=x_test.shape[1],
                      use_skip_connections=True,
                      regression=True,
                      dropout_rate=0.5,     
                      output_len=1)  

model11.fit(x_train, y_train, batch_size=256, epochs=10)   

(1555, 24, 17)
(1555,)
(653, 24, 17)
(653,)
(480, 24, 17)
x.shape= (None, 50)
model.x = (None, 24, 17)
model.y = (None, 1)
Epoch 1/10


C:\Users\wjd72\anaconda3\envs\tensorflow_env\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


7/7 [==============================] - 9s 543ms/step - loss: 26223922.0000
Epoch 2/10
7/7 [==============================] - 4s 587ms/step - loss: 277202.0000
Epoch 3/10
7/7 [==============================] - 4s 624ms/step - loss: 31139.8613
Epoch 4/10
7/7 [==============================] - 4s 572ms/step - loss: 7608.5659
Epoch 5/10
7/7 [==============================] - 4s 588ms/step - loss: 2526.7625
Epoch 6/10
7/7 [==============================] - 4s 594ms/step - loss: 1108.3409
Epoch 7/10
7/7 [==============================] - 5s 681ms/step - loss: 562.5005
Epoch 8/10
7/7 [==============================] - 4s 605ms/step - loss: 356.1104
Epoch 9/10
7/7 [==============================] - 4s 614ms/step - loss: 246.2487
Epoch 10/10
7/7 [==============================] - 5s 652ms/step - loss: 179.4387


In [35]:
p = model11.predict(x_validation)
p = p.flatten()

h1 = []
for i in range(int(len(p)/24)): 
    h1.append(np.sum(p[24*i:24*(i+1)]))    
    
    
h1.insert(0,0)     # 한 번만 클릭!
h11 = h1[4:14] + h1[18:21]
     
submission1['heat_supply_day1'] = h11
submission1 

,yy,mm,dd,heat_supply_day1,heat_supply_day2
0,2022,3,4,307.457581,NaN
1,2022,3,5,372.321442,NaN
2,2022,3,6,402.689148,NaN
3,2022,3,7,410.320312,NaN
4,2022,3,8,353.249573,NaN
5,2022,3,9,345.068573,NaN
6,2022,3,10,343.239014,NaN
7,2022,3,11,351.410034,NaN
8,2022,3,12,364.717804,NaN
9,2022,3,13,399.322510,NaN


In [30]:
h = []
for i in range(int(len(p)/24)): 
    h.append(p[24*i:24*(i+1)])
    
df = pd.DataFrame(h)
df1 = df.T
df1.columns = pd.date_range('2022-03-01', periods= 20, freq='1D')
df1

,2022-03-01,2022-03-02,2022-03-03,2022-03-04,2022-03-05,2022-03-06,2022-03-07,2022-03-08,2022-03-09,2022-03-10,2022-03-11,2022-03-12,2022-03-13,2022-03-14,2022-03-15,2022-03-16,2022-03-17,2022-03-18,2022-03-19,2022-03-20
0,18.972025,13.299316,14.432197,14.808496,15.891197,20.471695,19.376097,16.832682,20.714352,22.381336,19.118418,21.186453,17.866720,16.250013,15.673528,19.433569,15.384424,16.396431,13.826692,19.095909
1,16.017933,13.310829,14.469979,14.858277,13.946581,20.081163,20.305317,20.492119,15.400646,19.520464,19.737026,20.454212,17.941931,17.057764,15.610044,18.922094,15.393628,16.488386,14.706009,18.619707
2,17.310566,13.325716,14.506656,17.042643,15.136404,22.244400,19.718426,12.186537,20.378897,18.469864,20.385984,22.151360,18.570026,16.294529,15.501823,17.532305,15.388602,16.586779,21.076908,19.464769
3,17.456446,13.361415,14.538879,16.463385,16.448961,24.954834,24.182627,19.479034,22.174109,20.064590,21.198013,15.615176,20.422615,15.771242,17.944571,20.035240,15.376842,16.762167,19.011806,21.477659
4,18.325102,13.407391,14.548820,13.491493,17.461527,24.057655,26.419491,21.305521,17.859308,19.737459,21.597673,20.007711,17.667763,15.096271,14.320565,20.732992,15.371013,16.500301,17.042156,20.951109
5,18.683201,13.481912,14.545159,19.162533,19.194897,25.353437,27.645384,19.340799,18.881969,21.196688,18.963213,19.947069,19.957495,15.183195,15.285352,20.161200,15.374804,16.171640,17.398201,19.895115
6,18.976870,13.549552,14.539135,19.040144,19.027128,23.561188,22.769356,20.315119,20.114180,17.142658,19.071613,19.882944,20.465904,15.413031,17.469820,18.626095,15.375326,16.072884,20.355986,19.007792
7,16.232479,13.591860,14.534103,17.794737,17.598864,18.289917,23.015598,20.794180,18.160469,19.776047,19.282330,20.824997,19.634642,15.400781,14.369184,18.205860,15.393543,16.388498,19.171406,17.664312
8,15.634671,13.622545,14.529170,11.196826,16.528557,12.380858,22.543173,19.252989,12.099239,13.847053,13.779226,18.686941,15.826606,15.594422,11.650359,17.082851,15.435310,16.259357,12.039336,11.429593
9,16.393341,13.630454,14.521430,10.796200,17.529613,10.341463,16.069853,13.149328,8.878242,13.440184,14.227680,14.076374,12.622230,15.735998,8.762494,14.906001,15.484247,15.309722,8.402020,10.049915


In [31]:
df1.to_csv('day1_TCN-50_3_0.5_256.csv', index=False)

# 모델 2(절감 모델)

In [24]:
X = training.drop(['일자','smart_farm.heat_supply'], axis=1)
y = training[['smart_farm.heat_supply']].values

training_data, test_data, y_tr, y_te = train_test_split(X, y, test_size=0.3, shuffle=False)  

validation = testing.drop(['일자','smart_farm.heat_supply'], axis=1)


y_tr1 = y_tr.flatten()
y_te1 = y_te.flatten()


x_train = []
y_train = []

for i in range(24, training_data.shape[0]):
  x_train.append(training_data[i-24:i])
  y_train.append(y_tr1[i])
    
x_test = []
y_test = []

for i in range(24, test_data.shape[0]): 
  x_test.append(test_data[i-24:i])
  y_test.append(y_te1[i])
x_train, y_train = np.array(x_train), np.array(y_train)
x_test, y_test = np.array(x_test), np.array(y_test)



x_validation = []
for i in range(24, validation.shape[0]):
  x_validation.append(validation[i-24:i])
x_validation = np.array(x_validation)


print(x_train.shape) 
print(y_train.shape) 
print(x_test.shape)
print(y_test.shape)
print(x_validation.shape)

model11 = compiled_tcn(return_sequences=False,
                      num_feat=x_test.shape[2],
                      nb_filters=50,    
                      num_classes=0,
                      kernel_size=3,     
                      dilations=[2 ** i for i in range(9)],
                      nb_stacks=1,
                      max_len=x_test.shape[1],
                      use_skip_connections=True,
                      regression=True,
                      dropout_rate=0.5,     
                      output_len=1)  

model11.fit(x_train, y_train, batch_size=256, epochs=10)   

(1555, 24, 21)
(1555,)
(653, 24, 21)
(653,)
(480, 24, 21)
x.shape= (None, 50)
model.x = (None, 24, 21)
model.y = (None, 1)
Epoch 1/10


C:\Users\wjd72\anaconda3\envs\tensorflow_env\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


7/7 [==============================] - 9s 588ms/step - loss: 14436473.0000
Epoch 2/10
7/7 [==============================] - 4s 575ms/step - loss: 143011.8750
Epoch 3/10
7/7 [==============================] - 4s 605ms/step - loss: 16717.5117
Epoch 4/10
7/7 [==============================] - 4s 560ms/step - loss: 3496.4597
Epoch 5/10
7/7 [==============================] - 4s 617ms/step - loss: 1051.8151
Epoch 6/10
7/7 [==============================] - 5s 648ms/step - loss: 504.8039
Epoch 7/10
7/7 [==============================] - 4s 554ms/step - loss: 295.4888
Epoch 8/10
7/7 [==============================] - 4s 554ms/step - loss: 214.3126
Epoch 9/10
7/7 [==============================] - 4s 607ms/step - loss: 170.8355
Epoch 10/10
7/7 [==============================] - 5s 696ms/step - loss: 164.8398


In [36]:
p2 = model11.predict(x_validation)  
p2 = p2.flatten()

h2 = []
for i in range(int(len(p2)/24)): 
    h2.append(np.sum(p2[24*i:24*(i+1)]))
h2.insert(0,0)     # 한 번만 클릭!
h22 = h2[4:14] + h2[18:21]

submission1['heat_supply_day2'] = h22
submission1   

,yy,mm,dd,heat_supply_day1,heat_supply_day2
0,2022,3,4,307.457581,283.354553
1,2022,3,5,372.321442,330.922516
2,2022,3,6,402.689148,360.670166
3,2022,3,7,410.320312,387.759125
4,2022,3,8,353.249573,382.496521
5,2022,3,9,345.068573,357.698486
6,2022,3,10,343.239014,310.415527
7,2022,3,11,351.410034,276.409271
8,2022,3,12,364.717804,325.456177
9,2022,3,13,399.322510,420.248840


In [32]:
h = []
for i in range(int(len(p2)/24)): 
    h.append(p2[24*i:24*(i+1)])
    
df = pd.DataFrame(h)
df1 = df.T
df1.columns = pd.date_range('2022-03-01', periods= 20, freq='1D')
df1

,2022-03-01,2022-03-02,2022-03-03,2022-03-04,2022-03-05,2022-03-06,2022-03-07,2022-03-08,2022-03-09,2022-03-10,2022-03-11,2022-03-12,2022-03-13,2022-03-14,2022-03-15,2022-03-16,2022-03-17,2022-03-18,2022-03-19,2022-03-20
0,12.604116,11.641474,12.467013,14.963728,13.463450,16.817398,21.439301,18.435415,20.049826,19.732611,15.255812,13.713097,15.105820,14.988790,14.273976,18.316673,12.517156,15.568018,16.574003,17.912634
1,13.076361,11.598669,12.544522,15.095373,13.096450,16.963133,17.280706,21.217081,16.971416,18.117641,11.276168,15.253617,13.396463,14.157926,14.281866,18.187992,12.542837,15.744192,14.476195,16.472576
2,14.007628,11.537972,12.624168,16.004370,12.795797,16.600019,15.737562,18.430140,19.122881,15.235947,14.044804,18.637423,15.242239,15.329225,14.742507,17.683561,12.536285,18.169207,18.588312,17.609732
3,15.274596,11.474612,12.708916,12.190719,13.443456,18.797472,18.093706,21.323864,18.313070,17.483152,17.073305,13.441393,15.455646,15.271762,17.139864,18.088221,12.529853,18.433290,18.751204,17.434162
4,17.093306,11.419968,12.793873,14.239297,13.727919,19.685699,21.921362,17.576212,18.800793,18.863192,16.692543,13.138276,14.492353,15.777905,14.006132,18.656832,12.574410,18.400604,19.050226,19.413486
5,15.439793,11.363601,12.875841,18.125870,16.009813,19.365526,20.530077,17.138514,17.571371,18.907301,16.029507,16.254173,16.436354,15.557261,14.930018,18.970860,12.624985,18.273830,18.189474,17.604515
6,15.413546,11.317633,12.961370,18.003716,16.125587,19.953003,20.339266,16.251490,20.099083,18.437578,15.582887,17.394627,16.611210,15.237901,16.013422,20.279062,12.681150,17.883345,19.591679,18.434416
7,11.771573,11.301970,13.046881,17.593613,14.188059,16.541426,19.095581,16.329281,18.373196,19.503454,15.578693,19.290024,15.169267,14.999990,13.976953,16.986345,12.774556,18.014223,17.688986,16.273451
8,11.720160,11.336064,13.143240,9.872619,7.955261,8.143238,15.271071,12.408072,7.923091,8.983728,6.374693,12.546515,12.405035,15.377532,4.399416,13.510367,12.869948,16.776171,15.966140,7.742380
9,11.456092,11.390734,13.236931,8.359399,8.603927,5.648925,7.525070,8.420144,4.562203,11.333586,10.017457,13.818700,17.868011,15.466933,4.357753,12.480086,12.989300,17.500519,17.438154,4.962015


In [33]:
df1.to_csv('day2_TCN-50_3_0.5_256.csv', index=False)

In [34]:
submission1.to_csv('220224.csv', index=False) 